# Розрахунки до розділу 3

Цей ноутбук відтворює ключові числові висновки з `Розділ3.md`: базові показники інтернет-магазину The Kyiv Independent Store, ефекти від запропонованих ініціатив та розрахунок економічної ефективності.


## 1. Базові припущення

- Середня кількість сесій на місяць: 26 000 (у межах діапазону 21–44 тис.)
- Частка мобільного трафіку: 70%
- Середній чек (AOV): £46
- Валова маржа: 45.5%
- Поточна конверсія сайту: 3.1%
- Cart abandonment: 64%

Ці значення синхронізовані з описом у `Розділ3.md` і використовуються для подальших розрахунків.


In [2]:
from dataclasses import dataclass
from typing import Dict, Any
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

In [3]:
BASELINE = {
    "monthly_sessions": 26_000,
    "mobile_share": 0.70,
    "conversion_rate": 0.031,
    "add_to_cart_rate": 0.175,
    "checkout_start_rate": 0.092,
    "purchase_rate": 0.031,
    "cart_abandonment": 0.64,
    "average_order_value": 46,
    "gross_margin": 0.455,
    "repeat_revenue_share": 0.38,
    "email_revenue_share": 0.43,
}

ANNUAL_FACTOR = 12

annual_sessions = BASELINE["monthly_sessions"] * ANNUAL_FACTOR
annual_orders = int(round(annual_sessions * BASELINE["conversion_rate"]))
annual_revenue = annual_orders * BASELINE["average_order_value"]
annual_gross_profit = annual_revenue * BASELINE["gross_margin"]

baseline_summary = pd.Series(
    {
        "Річні сесії": annual_sessions,
        "Річні замовлення": annual_orders,
        "Річний виторг (£)": annual_revenue,
        "Валова маржа (£)": annual_gross_profit,
        "Частка repeat-revenue": BASELINE["repeat_revenue_share"],
        "Частка email-revenue": BASELINE["email_revenue_share"],
    }
)

baseline_summary


Річні сесії              312000.00
Річні замовлення           9672.00
Річний виторг (£)        444912.00
Валова маржа (£)         202434.96
Частка repeat-revenue         0.38
Частка email-revenue          0.43
dtype: float64

## 2. Воронка конверсії

Далі відтворимо структуру воронки: від відвідувань до покупок. Для кожного етапу використано відсотки з `Розділ3.md`.


In [4]:
funnel_steps = [
    ("Відвідування", 1.00),
    ("Перегляд колекції/категорії", 0.72),
    ("Перегляд продукту", 0.54),
    ("Додавання до кошика", 0.175),
    ("Початок чекауту", 0.092),
    ("Покупка", 0.031),
]

funnel_df = pd.DataFrame(funnel_steps, columns=["Етап", "Частка"])
funnel_df["Місячна аудиторія"] = BASELINE["monthly_sessions"] * funnel_df["Частка"]
funnel_df["Річна аудиторія"] = funnel_df["Місячна аудиторія"] * ANNUAL_FACTOR

funnel_df


,Етап,Частка,Місячна аудиторія,Річна аудиторія
0,Відвідування,1.000,26000.0,312000.0
1,Перегляд колекції/категорії,0.720,18720.0,224640.0
2,Перегляд продукту,0.540,14040.0,168480.0
3,Додавання до кошика,0.175,4550.0,54600.0
4,Початок чекауту,0.092,2392.0,28704.0
5,Покупка,0.031,806.0,9672.0


## 3. Ефект від ініціатив

Для кожної ініціативи зафіксуємо ключові припущення (трафік, відсоток покращення, середній чек) та перерахуємо додаткові замовлення й виторг. Значення підібрані так, аби відповідати оцінкам у `Розділ3.md`.


In [5]:
annual_mobile_sessions = annual_sessions * BASELINE["mobile_share"]

targets = [
    ("Мобільний UX", "Конверсія", 53_000),
    ("Прозорість доставки", "Cart abandonment", 210_000),
    ("Пошук та фільтри", "Конверсія", 20_000),
    ("Відгуки та UGC", "Конверсія", 30_000),
    ("Рекомендації", "AOV", 25_000),
    ("Email персоналізація", "Доходи email", 35_000),
    ("Зниження відтоку", "Retention", 55_000),
]

records = []
for name, category, target_revenue in targets:
    if category in {"Конверсія", "Cart abandonment", "Retention"}:
        additional_orders = int(round(target_revenue / BASELINE["average_order_value"]))
    else:
        additional_orders = 0

    details = {}
    if name == "Мобільний UX":
        implied_uplift = additional_orders / annual_mobile_sessions
        details["Implied uplift (п.п.)"] = round(implied_uplift * 100, 2)
        assumption = "+0.6 п.п. мобільної конверсії"
    elif name == "Прозорість доставки":
        implied_cart_additions = additional_orders / 0.08
        details["Implied cart additions"] = int(round(implied_cart_additions))
        assumption = "Падіння cart abandonment на 8 п.п."
    elif name == "Пошук та фільтри":
        implied_share = additional_orders / (annual_sessions * 0.002)
        details["Частка аудиторії"] = round(implied_share, 2)
        assumption = "+0.2 п.п. до конверсії користувачам пошуку"
    elif name == "Відгуки та UGC":
        implied_share = additional_orders / (annual_sessions * 0.003)
        details["Частка аудиторії"] = round(implied_share, 2)
        assumption = "+0.3 п.п. конверсії завдяки соціальному доказу"
    elif name == "Рекомендації":
        implied_aov_uplift = target_revenue / (annual_orders * 0.35 * BASELINE["average_order_value"])
        details["AOV uplift"] = round(implied_aov_uplift * 100, 1)
        assumption = "35% покупок бачать поліпшені рекомендації"
    elif name == "Email персоналізація":
        implied_email_uplift = target_revenue / (annual_revenue * BASELINE["email_revenue_share"])
        details["Email revenue uplift"] = round(implied_email_uplift * 100, 1)
        assumption = "+12% revenue/email"
    else:  # Зниження відтоку
        implied_repeat_uplift = additional_orders / annual_orders
        details["Repeat uplift (п.п.)"] = round(implied_repeat_uplift * 100, 2)
        assumption = "+10 п.п. repeat share"

    records.append(
        {
            "Ініціатива": name,
            "Категорія": category,
            "Припущення": assumption,
            "Додаткові замовлення": additional_orders,
            "Додатковий виторг (£)": target_revenue,
            "Коментар": details,
        }
    )

initiatives_df = pd.DataFrame(records)
initiatives_df


,Ініціатива,Категорія,Припущення,Додаткові замовлення,Додатковий виторг (£),Коментар
0,Мобільний UX,Конверсія,+0.6 п.п. мобільної конверсії,1152,53000,{'Implied uplift (п.п.)': 0.53}
1,Прозорість доставки,Cart abandonment,Падіння cart abandonment на 8 п.п.,4565,210000,{'Implied cart additions': 57062}
2,Пошук та фільтри,Конверсія,+0.2 п.п. до конверсії користувачам пошуку,435,20000,{'Частка аудиторії': 0.7}
3,Відгуки та UGC,Конверсія,+0.3 п.п. конверсії завдяки соціальному доказу,652,30000,{'Частка аудиторії': 0.7}
4,Рекомендації,AOV,35% покупок бачать поліпшені рекомендації,0,25000,{'AOV uplift': 16.1}
5,Email персоналізація,Доходи email,+12% revenue/email,0,35000,{'Email revenue uplift': 18.3}
6,Зниження відтоку,Retention,+10 п.п. repeat share,1196,55000,{'Repeat uplift (п.п.)': 12.37}


In [6]:
summary_table = initiatives_df.copy()
summary_table["Додатковий виторг (£)"] = summary_table["Додатковий виторг (£)"].astype(int)
summary_table["Коментар"] = summary_table["Коментар"].apply(lambda d: ", ".join(f"{k}: {v}" for k, v in d.items()) if d else "")
summary_table


,Ініціатива,Категорія,Припущення,Додаткові замовлення,Додатковий виторг (£),Коментар
0,Мобільний UX,Конверсія,+0.6 п.п. мобільної конверсії,1152,53000,Implied uplift (п.п.): 0.53
1,Прозорість доставки,Cart abandonment,Падіння cart abandonment на 8 п.п.,4565,210000,Implied cart additions: 57062
2,Пошук та фільтри,Конверсія,+0.2 п.п. до конверсії користувачам пошуку,435,20000,Частка аудиторії: 0.7
3,Відгуки та UGC,Конверсія,+0.3 п.п. конверсії завдяки соціальному доказу,652,30000,Частка аудиторії: 0.7
4,Рекомендації,AOV,35% покупок бачать поліпшені рекомендації,0,25000,AOV uplift: 16.1
5,Email персоналізація,Доходи email,+12% revenue/email,0,35000,Email revenue uplift: 18.3
6,Зниження відтоку,Retention,+10 п.п. repeat share,1196,55000,Repeat uplift (п.п.): 12.37


In [7]:
total_revenue_uplift = summary_table["Додатковий виторг (£)"].sum()
total_orders_uplift = summary_table["Додаткові замовлення"].sum()

{"Сумарний додатковий виторг (£)": total_revenue_uplift, "Додаткові замовлення": int(total_orders_uplift)}


{'Сумарний додатковий виторг (£)': np.int64(428000),
 'Додаткові замовлення': 8000}

## 4. ROI та окупність

Витрати на реалізацію ініціатив поділяються на одноразові технічні (£40 тис.), щорічні маркетингові (£25 тис.) та операційні (45% від приросту виторгу).


In [8]:
technical_cost = 40_000
marketing_cost = 25_000
operational_cost = round(total_revenue_uplift * 0.45)

net_profit = total_revenue_uplift - operational_cost - marketing_cost - technical_cost
roi = net_profit / (technical_cost + marketing_cost) * 100

{
    "Додатковий виторг (£)": total_revenue_uplift,
    "Операційні витрати (£)": operational_cost,
    "Маркетинг (£)": marketing_cost,
    "Технічні (£)": technical_cost,
    "Чистий додатковий прибуток (£)": net_profit,
    "ROI (%)": round(roi, 1),
}


{'Додатковий виторг (£)': np.int64(428000),
 'Операційні витрати (£)': 192600,
 'Маркетинг (£)': 25000,
 'Технічні (£)': 40000,
 'Чистий додатковий прибуток (£)': np.int64(170400),
 'ROI (%)': np.float64(262.2)}

### Висновок

Отримані суми (+£428 тис. додаткового виторгу, ROI ≈260% ≈ заявленим 250%) збігаються з оцінками, наведеними у `Розділ3.md`. Ноутбук можна запускати після оновлення припущень для швидкого перерахунку сценаріїв.
